# data script

In [108]:
import os
import json

In [ ]:
root='./data'
name='train_wiki'
path = os.path.join(root, name + ".json")
json_data = json.load(open(path))
classes = list(json_data.keys()) #64 classes

In [109]:
root='./data'
val_dataset='val_wiki'
path = os.path.join(root, val_dataset + ".json")
json_data = json.load(open(path))
classes = list(json_data.keys()) #16 classes

In [111]:
print(len(classes))

16


In [49]:
classid=50
print(classes[classid])
classdata=json_data[classes[classid]]
print('class length:',len(classdata)) # 700 instances
instanceid=2
for key in classdata[instanceid]:
    print(key)
    print('\t',end=' ')
    for name in classdata[instanceid][key]:
        print(name,end=' ')
    print()
print(classdata[instanceid]['tokens'][0])

P460
class length: 700
tokens
	 Santa Luzia ( Portuguese for Saint Lucia ) is a civil parish in the municipality of São Roque do Pico in the Azores . 
h
	 santa luzia Q2147595 [[0, 1]] 
t
	 saint lucia Q398922 [[5, 6]] 
Santa


In [51]:
classid=50
class_name=list(json_data.keys())[classid]
j=2
item=json_data[class_name][j]
print(item['tokens'])
print(item['h'][2][0],item['t'][2][0])
raw_tokens, pos_head, pos_tail=item['tokens'],item['h'][2][0],item['t'][2][0]

['Santa', 'Luzia', '(', 'Portuguese', 'for', 'Saint', 'Lucia', ')', 'is', 'a', 'civil', 'parish', 'in', 'the', 'municipality', 'of', 'São', 'Roque', 'do', 'Pico', 'in', 'the', 'Azores', '.']
[0, 1] [5, 6]


In [53]:
max_length=128
indexed_tokens = [] # index in dictionary for each tokens in the sentence

word2id = json.load(open('./pretrain/glove/glove_word2id.json')) # dict 40w词语包括标点符号、停用词、 [UNK], [PAD]
for token in raw_tokens:
    token = token.lower()
    if token in word2id:
        indexed_tokens.append(word2id[token])
    else:
        indexed_tokens.append(word2id['[UNK]'])

# padding
while len(indexed_tokens) < max_length: # padding words to max_length
    indexed_tokens.append(word2id['[PAD]'])
indexed_tokens = indexed_tokens[:max_length]

# pos
pos1 = np.zeros((max_length), dtype=np.int32)
pos2 = np.zeros((max_length), dtype=np.int32)
pos1_in_index = min(max_length, pos_head[0]) # the index of the first token of head entity
pos2_in_index = min(max_length, pos_tail[0])# the index of the first token of tail entity
for i in range(max_length):
    pos1[i] = i - pos1_in_index + max_length
    pos2[i] = i - pos2_in_index + max_length

# mask
mask = np.zeros((max_length), dtype=np.int32)
mask[:len(indexed_tokens)] = 1


In [63]:
word, pos1, pos2, mask =indexed_tokens, pos1, pos2, mask
print(len(word),word)
print(len(pos1),pos1-128)
print(len(pos2),pos2-128)
print(len(mask),mask)

128 [2669, 156976, 23, 3578, 10, 2759, 14418, 24, 14, 7, 786, 3363, 6, 0, 2475, 3, 11531, 20228, 88, 31710, 6, 0, 26106, 2, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001]
128 [  0   1   2   3   4   5   6   7   8   9

In [23]:
N, K, Q=5,5,5 # N-way K-supportShot Q-queryShot
na_rate=0

support_set = {'word': [], 'pos1': [], 'pos2': [], 'mask': [] }
query_set = {'word': [], 'pos1': [], 'pos2': [], 'mask': [] }
query_label = []

target_classes = random.sample(classes,N) # N classes from 64
for i, class_name in enumerate(target_classes):
    indices = np.random.choice(  list(range(len(self.json_data[class_name]))),   K +  Q, False) # K+Q instances from 700 
    count = 0
    for j in indices:
        word, pos1, pos2, mask = self.__getraw__(
                self.json_data[class_name][j])
        word = torch.tensor(word).long()
        pos1 = torch.tensor(pos1).long()
        pos2 = torch.tensor(pos2).long()
        mask = torch.tensor(mask).long()
        if count < self.K:
            self.__additem__(support_set, word, pos1, pos2, mask)
        else:
            self.__additem__(query_set, word, pos1, pos2, mask)
        count += 1

    query_label += [i] * self.Q
return support_set, query_set, query_label

# model - sentence encoder

In [2]:
# standford GloVe word embedding, 400K+2 vocab，50 embedding
# 包括标点符号、停用词，最后两个为[UNK]、[PAD]
# 最后一个vocab为[PAD]，pad的embeddding是全部为0的向量

import numpy as np
import json

glove_mat = np.load('./pretrain/glove/glove_mat.npy') #  numpy.ndarray (400002, 50)
glove_word2id = json.load(open('./pretrain/glove/glove_word2id.json')) # dict 40w词语包括标点符号、停用词、 [UNK], [PAD]

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math

class Embedding(nn.Module): #词语及position的embedding

    def __init__(self, word_vec_mat, max_length, word_embedding_dim=50, pos_embedding_dim=5):
        nn.Module.__init__(self)

        self.max_length = max_length  # 128
        self.word_embedding_dim = word_embedding_dim #50
        self.pos_embedding_dim = pos_embedding_dim #5
        
        # Word embedding
        # unk = torch.randn(1, word_embedding_dim) / math.sqrt(word_embedding_dim)
        # blk = torch.zeros(1, word_embedding_dim)
        word_vec_mat = torch.from_numpy(word_vec_mat) #
        self.word_embedding = nn.Embedding(word_vec_mat.shape[0], self.word_embedding_dim, padding_idx=word_vec_mat.shape[0] - 1)
        self.word_embedding.weight.data.copy_(word_vec_mat) #torch.nn.embedding的weight即为词典各词语的编码

        # Position Embedding
        self.pos1_embedding = nn.Embedding(2 * max_length, pos_embedding_dim, padding_idx=0)# 256个position 编码维度为5维 第0个词为pad
        self.pos2_embedding = nn.Embedding(2 * max_length, pos_embedding_dim, padding_idx=0)

    def forward(self, inputs):
        word = inputs['word']
        pos1 = inputs['pos1']
        pos2 = inputs['pos2']
        
        x = torch.cat([self.word_embedding(word), 
                            self.pos1_embedding(pos1), 
                            self.pos2_embedding(pos2)], 2)
        return x

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from torch import optim

class Encoder(nn.Module):
    def __init__(self, max_length, word_embedding_dim=50, pos_embedding_dim=5, hidden_size=230):
        nn.Module.__init__(self)

        self.max_length = max_length
        self.hidden_size = hidden_size
        self.embedding_dim = word_embedding_dim + pos_embedding_dim * 2
        self.conv = nn.Conv1d(self.embedding_dim, self.hidden_size, 3, padding=1)  #1层CNN呀？
        self.pool = nn.MaxPool1d(max_length)

        # For PCNN
        self.mask_embedding = nn.Embedding(4, 3) ###########？？
        self.mask_embedding.weight.data.copy_(torch.FloatTensor([[1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 0]]))
        self.mask_embedding.weight.requires_grad = False
        self._minus = -100

    def forward(self, inputs):
        return self.cnn(inputs)

    def cnn(self, inputs):
        x = self.conv(inputs.transpose(1, 2))
        x = F.relu(x)
        x = self.pool(x)
        return x.squeeze(2) # n x hidden_size

    def pcnn(self, inputs, mask):
        x = self.conv(inputs.transpose(1, 2)) # n x hidden x length
        mask = 1 - self.mask_embedding(mask).transpose(1, 2) # n x 3 x length
        pool1 = self.pool(F.relu(x + self._minus * mask[:, 0:1, :]))
        pool2 = self.pool(F.relu(x + self._minus * mask[:, 1:2, :]))
        pool3 = self.pool(F.relu(x + self._minus * mask[:, 2:3, :]))
        x = torch.cat([pool1, pool2, pool3], 1)
        x = x.squeeze(2) # n x (hidden_size * 3) 



In [5]:
import torch.nn as nn
class CNNSentenceEncoder(nn.Module):

    def __init__(self, word_vec_mat, word2id, max_length, word_embedding_dim=50, 
            pos_embedding_dim=5, hidden_size=230):
        nn.Module.__init__(self)
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.embedding = Embedding(word_vec_mat, max_length, word_embedding_dim, pos_embedding_dim)
        self.encoder = Encoder(max_length, word_embedding_dim, pos_embedding_dim, hidden_size) 
        self.word2id = word2id

    def forward(self, inputs):
        x = self.embedding(inputs)  #[200, 128, 60]
        x = self.encoder(x)  #[200, 230]
        return x

    def tokenize(self, raw_tokens, pos_head, pos_tail): # sentence tokens, head entity tokens index, tail entity tokens index
        # token -> index
        indexed_tokens = [] # index in dictionary for each tokens in the sentence
        for token in raw_tokens:
            token = token.lower()
            if token in self.word2id:
                indexed_tokens.append(self.word2id[token])
            else:
                indexed_tokens.append(self.word2id['[UNK]'])
        
        # padding
        while len(indexed_tokens) < self.max_length: # padding words to max_length
            indexed_tokens.append(self.word2id['[PAD]'])
        indexed_tokens = indexed_tokens[:self.max_length]

        # pos
        pos1 = np.zeros((self.max_length), dtype=np.int32)
        pos2 = np.zeros((self.max_length), dtype=np.int32)
        pos1_in_index = min(self.max_length, pos_head[0]) # the index of the first token of head entity
        pos2_in_index = min(self.max_length, pos_tail[0])# the index of the first token of tail entity
        for i in range(self.max_length):
            pos1[i] = i - pos1_in_index + self.max_length
            pos2[i] = i - pos2_in_index + self.max_length

        # mask
        mask = np.zeros((self.max_length), dtype=np.int32)
        mask[:len(indexed_tokens)] = 1

        return indexed_tokens, pos1, pos2, mask

In [6]:
sentence_encoder = CNNSentenceEncoder(glove_mat, glove_word2id,  max_length=128)

# data

In [7]:
from fewshot_re_kit.data_loader import get_loader
trainname='train_wiki'
trainN,K,Q,na_rate,batch_size=10,5,6,0,4

train_data_loader = get_loader(trainname, sentence_encoder,
                N=trainN, K=K, Q=Q, na_rate=na_rate, batch_size=batch_size,num_workers=0)
support, query, label = next(train_data_loader)

In [8]:
print(support['word'].shape,support['pos1'].shape,support['pos2'].shape)
print(query['word'].shape)
print(label.shape)

torch.Size([200, 128]) torch.Size([200, 128]) torch.Size([200, 128])
torch.Size([240, 128])
torch.Size([240])


In [114]:
val_dataset='val_wiki'
evalN=5  
val_data_loader = get_loader(val_dataset, sentence_encoder,
                N=evalN, K=K, Q=Q, na_rate= na_rate, batch_size=batch_size,num_workers=0)
support, query, label = next(val_data_loader)

In [115]:
print(support['word'].shape,support['pos1'].shape,support['pos2'].shape)
print(query['word'].shape)
print(label.shape)

torch.Size([100, 128]) torch.Size([100, 128]) torch.Size([100, 128])
torch.Size([120, 128])
torch.Size([120])


# model script - embedding, encoder

In [58]:
# Embedding
max_length=128
word_vec_mat = torch.from_numpy(glove_mat)
word_embedding_dim=50
pos_embedding_dim=5
word,pos1,pos2 = support['word'],support['pos1'],support['pos2']

word_embedding = nn.Embedding(word_vec_mat.shape[0], word_embedding_dim, padding_idx=word_vec_mat.shape[0] - 1) 
#word_vec_mat.shape[0], 50,400001
word_embedding.weight.data.copy_(word_vec_mat) #torch.nn.embedding的weight即为词典各词语的编码      
pos1_embedding = nn.Embedding(2 * max_length, pos_embedding_dim, padding_idx=0)# 256个position 编码维度为5维 第0个词为pad
pos2_embedding = nn.Embedding(2 * max_length, pos_embedding_dim, padding_idx=0)

worde=word_embedding(word)
pos1e=pos1_embedding(pos1)
pos2e=pos2_embedding(pos2)
x=torch.cat([worde,pos1e,pos2e], 2) #[200, 128, 60]

In [60]:
print(word.shape,pos1.shape,pos2.shape)
print(worde.shape,pos1e.shape,pos2e.shape)
print(x.shape)

torch.Size([200, 128]) torch.Size([200, 128]) torch.Size([200, 128])
torch.Size([200, 128, 50]) torch.Size([200, 128, 5]) torch.Size([200, 128, 5])
torch.Size([200, 128, 60])


In [61]:
# Encoder
hidden_size=230

embedding_dim = word_embedding_dim + pos_embedding_dim * 2
conv = nn.Conv1d(embedding_dim, hidden_size, 3, padding=1)  #1层CNN呀？
pool = nn.MaxPool1d(max_length)

x1 = conv(x.transpose(1, 2))
x2 = F.relu(x1)
x3 = pool(x2)
support_emb=x3.squeeze(2)

In [62]:
print(x1.shape)
print(x2.shape)
print(x3.shape)
print(support_emb.shape)  #[200, 230]

torch.Size([200, 230, 128])
torch.Size([200, 230, 128])
torch.Size([200, 230, 1])
torch.Size([200, 230])


# model script -- proto

In [10]:
hidden_size=230

drop = nn.Dropout()

support_emb = sentence_encoder(support) # #[200,230] (Batch_size * N * K, hidden_size) 
query_emb = sentence_encoder(query) #[240, 230] [N*Q*batchsize,hidden_size]
support_dr = drop(support_emb)
query_dr = drop(query_emb)  #[240, 230] 

support_vi = support_dr.view(-1, trainN, K, hidden_size) #[4, 10, 5, 230] (Batch_size, N, K, hidden_size)
support_me = torch.mean(support_vi, 2) # [4, 10, 230], [batch_size,N,hidden_size] 关于K-shot平均后各类的原型 Calculate prototype for each class
x=support_me.unsqueeze(1) #[4, 1, 10, 230]

total_Q = Q *trainN + na_rate * Q #60
query_vi = query_dr.view(-1, total_Q, hidden_size) # [4, 60, 230]
y=query_vi.unsqueeze(2) #[4, 60, 1, 230]

In [11]:
print(support_emb.shape,query_emb.shape)
print(support_vi.shape, support_me.shape,x.shape)
print(query_vi.shape,y.shape)

torch.Size([200, 230]) torch.Size([240, 230])
torch.Size([4, 10, 5, 230]) torch.Size([4, 10, 230]) torch.Size([4, 1, 10, 230])
torch.Size([4, 60, 230]) torch.Size([4, 60, 1, 230])


In [12]:
dim=3
# dot_xy=x * y #[4, 60, 10, 230] [batchsize, N*Q, N, 230]
# logits=dot_xy.sum(dim) #[4, 60, 10] [batchsize, N*Q, N] 各批任务各类各query样本属于各N个类的评分
l2_xy=-(torch.pow(x - y, 2))
logits=l2_xy.sum(dim) #[4, 60, 10] [batchsize, N*Q, N] 各批任务各类各query样本属于各N个类的评分


minn, _ = logits.min(-1) #[4, 60]
nona_logits=minn.unsqueeze(2) -1 #[4, 60, 1]
logits_with_nona = torch.cat([logits,nona_logits ], 2) # [4, 60, 11] [batchsize, N*Q, N+1]
xx=logits_with_nona.view(-1, trainN+1) #[240, 11] [batchsize*N*Q,N+1]
_, pred = torch.max(xx, 1) #[240]  batchsize*N*Q个样本的分类预测标签   

In [94]:
print(dot_xy.shape,logits.shape)
print(minn.shape,nona_logits.shape,logits_with_nona.shape)
print(xx.shape,pred.shape)

torch.Size([4, 60, 10, 230]) torch.Size([4, 60, 10])
torch.Size([4, 60]) torch.Size([4, 60, 1]) torch.Size([4, 60, 11])
torch.Size([240, 11]) torch.Size([240])


In [13]:
print(l2_xy.shape,logits.shape)
print(minn.shape,nona_logits.shape,logits_with_nona.shape)
print(xx.shape,pred.shape)

torch.Size([4, 60, 10, 230]) torch.Size([4, 60, 10])
torch.Size([4, 60]) torch.Size([4, 60, 1]) torch.Size([4, 60, 11])
torch.Size([240, 11]) torch.Size([240])


# model proto

In [33]:
import torch
from torch import autograd, optim, nn
from torch.autograd import Variable
from torch.nn import functional as F

class Proto(nn.Module):
    
    def __init__(self, sentence_encoder, dot=False):
        nn.Module.__init__(self)
        self.sentence_encoder = nn.DataParallel(sentence_encoder)
        self.cost = nn.CrossEntropyLoss()
        # self.fc = nn.Linear(hidden_size, hidden_size)
        self.drop = nn.Dropout()
        self.dot = dot
        
    def loss(self, logits, label):
        N = logits.size(-1)
        return self.cost(logits.view(-1, N), label.view(-1))

    def accuracy(self, pred, label):
        return torch.mean((pred.view(-1) == label.view(-1)).type(torch.FloatTensor))

    def __dist__(self, x, y, dim):
        if self.dot:
            return (x * y).sum(dim)
        else:
            return -(torch.pow(x - y, 2)).sum(dim)

    def __batch_dist__(self, S, Q):
        return self.__dist__(S.unsqueeze(1), Q.unsqueeze(2), 3)
    

    def forward(self, support, query, N, K, total_Q):
        support_emb = self.sentence_encoder(support) # #[200,230] (B * N * K, D), where D is the hidden size
        query_emb = self.sentence_encoder(query) # (B * total_Q, D)
        hidden_size = support_emb.size(-1)
        support = self.drop(support_emb)
        query = self.drop(query_emb)
        support = support.view(-1, N, K, hidden_size) # (B, N, K, D)
        query = query.view(-1, total_Q, hidden_size) # (B, total_Q, D)
        B = support.size(0) # Batch size
        support = torch.mean(support, 2) # Calculate prototype for each class
        logits = self.__batch_dist__(support, query) # (B, total_Q, N)
        minn, _ = logits.min(-1)
        logits = torch.cat([logits, minn.unsqueeze(2) - 1], 2) # (B, total_Q, N + 1)
        _, pred = torch.max(logits.view(-1, N+1), 1)
        return logits, pred   

In [34]:
model = Proto(sentence_encoder, dot=True)
logits, pred  = model(support, query,  trainN, K, Q *trainN + na_rate * Q)
right = model.accuracy(pred, label)
grad_iter=1
loss = model.loss(logits, label) / float(grad_iter)

In [43]:
print(logits.shape)
print(pred.shape)
print(right)
print(loss)

torch.Size([4, 60, 11])
torch.Size([240])
tensor(0.1083)
tensor(6.2720, grad_fn=<DivBackward0>)


# run  - train

In [98]:
model_name = 'proto'
encoder_name = 'cnn'
trainset_name='train_wiki'
valset_name='val_wiki'
evalN=5 # trainN=10
prefix = '-'.join([model_name, encoder_name, trainset_name, valset_name, str(evalN), str(K)])
prefix += '-dot'

save_ckpt=  'checkpoint/{}.pth.tar'.format(prefix)

In [99]:
learning_rate=1e-1
weight_decay=1e-5
lr_step_size=20000
optimizer = optim.SGD(model.parameters(), learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size)

In [100]:
model.train()

Proto(
  (sentence_encoder): DataParallel(
    (module): CNNSentenceEncoder(
      (embedding): Embedding(
        (word_embedding): Embedding(400002, 50, padding_idx=400001)
        (pos1_embedding): Embedding(256, 5, padding_idx=0)
        (pos2_embedding): Embedding(256, 5, padding_idx=0)
      )
      (encoder): Encoder(
        (conv): Conv1d(60, 230, kernel_size=(3,), stride=(1,), padding=(1,))
        (pool): MaxPool1d(kernel_size=128, stride=128, padding=0, dilation=1, ceil_mode=False)
        (mask_embedding): Embedding(4, 3)
      )
    )
  )
  (cost): CrossEntropyLoss()
  (drop): Dropout(p=0.5, inplace=False)
)

In [106]:
import sys

train_iter=30#000
grad_iter=1
val_step=2#000
start_iter = 0

best_acc = 0
not_best_count = 0 # Stop training after several epochs without improvement.
iter_loss = 0.0
iter_loss_dis = 0.0
iter_right = 0.0
iter_right_dis = 0.0
iter_sample = 0.0
for it in range(start_iter, start_iter + train_iter):
    support, query, label = next(train_data_loader)  
    logits, pred  = model(support, query, trainN, K, Q * trainN + na_rate * Q)
    loss = model.loss(logits, label) / float(grad_iter)
    right = model.accuracy(pred, label)
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
    if it % grad_iter == 0:
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    iter_loss += loss.data.item()
    iter_right +=  right.data.item()
    iter_sample += 1
    sys.stdout.write('step: {0:4} | loss: {1:2.6f}, accuracy: {2:3.2f}%'.format(it + 1, iter_loss / iter_sample, 100 * iter_right / iter_sample) +'\r')
    sys.stdout.flush()

    if (it + 1) % val_step == 0: #2#000
#         acc = self.eval(model, Batch_size, N_for_eval, K, Q, val_iter, na_rate=na_rate, pair=pair)
#         model.train()
#         if acc > best_acc:
#              print('Best checkpoint')
#             torch.save({'state_dict': model.state_dict()}, save_ckpt)
#             best_acc = acc
        iter_loss = 0.
        iter_loss_dis = 0.
        iter_right = 0.
        iter_right_dis = 0.
        iter_sample = 0.

print("\n####################\n")
print("Finish training " + prefix)

step:   30 | loss: 2.332235, accuracy: 13.75%
####################

Finish training proto-cnn-train_wiki-val_wiki-5-5-dot


# run eval

In [116]:
model.eval()

Proto(
  (sentence_encoder): DataParallel(
    (module): CNNSentenceEncoder(
      (embedding): Embedding(
        (word_embedding): Embedding(400002, 50, padding_idx=400001)
        (pos1_embedding): Embedding(256, 5, padding_idx=0)
        (pos2_embedding): Embedding(256, 5, padding_idx=0)
      )
      (encoder): Encoder(
        (conv): Conv1d(60, 230, kernel_size=(3,), stride=(1,), padding=(1,))
        (pool): MaxPool1d(kernel_size=128, stride=128, padding=0, dilation=1, ceil_mode=False)
        (mask_embedding): Embedding(4, 3)
      )
    )
  )
  (cost): CrossEntropyLoss()
  (drop): Dropout(p=0.5, inplace=False)
)

In [118]:
val_iter=1000      

iter_right = 0.0
iter_sample = 0.0
with torch.no_grad():
    for it in range(val_iter):
        support, query, label = next(val_data_loader)
        logits, pred = model(support, query, evalN, K, Q * evalN + Q * na_rate)
        right = model.accuracy(pred, label)
        iter_right += right.data.item()
        iter_sample += 1
        sys.stdout.write('[EVAL] step: {0:4} | accuracy: {1:3.2f}%'.format(it + 1, 100 * iter_right / iter_sample) +'\r')
        sys.stdout.flush()
    print("")
acc=iter_right / iter_sample

[EVAL] step: 1000 | accuracy: 29.33%
